# OH- in water

Related JDFTx files:

 * [W15OH1-confining.in](data/W15OH1-confining.in)
 * [W15OH1-confining.out](data/W15OH1-confining.out)
 * [W15OH1-confining.n](data/W15OH1-confining.n)
 * [W15OH1-confining.fluidN_H2O_H](data/W15OH1-confining.fluidN_H2O_H)
 * [W15OH1-confining.fluidN_H2O_O](data/W15OH1-confining.fluidN_H2O_O)
 
Import stuff and read the binary files:

In [1]:
import warnings
warnings.filterwarnings('ignore')
from jdftx import *

o_0 = np.fromfile('data/W15OH1-confining.fluidN_H2O_O').reshape(140,140,140)
h_0 = np.fromfile('data/W15OH1-confining.fluidN_H2O_H').reshape(140,140,140)
n_0 = np.fromfile('data/W15OH1-confining.n').reshape(140,140,140)

a = 35*bohr_as_angstrom
dx = a/140
x = np.arange(0, a, dx); x+=a/2; x %= a; x-= a/2; y = x[:]; z = x[:]
from itertools import product
xyz = np.asarray(list(product(x,y,z))) # array of coordinates of size (140^3, 3)

Visualize the explicit atoms and the fluid structure. I am only visualizing the large values of site densities, and the `points` that represent the fluid are sized proportional to the site density, large points represent large peaks. You can change the thresold for the site densities and play with the image.

In [7]:
from IPython.html.widgets import *
from IPython import display
from chemview import enable_notebook, RepresentationViewer, MolecularViewer
enable_notebook()

tv = jdout('data/W15OH1-confining').o.vis()

oSize, hSize = 10, 10
oxygen, hydrogen = None, None

@interact(O_threshold=(o_0.mean(), o_0.mean()+o_0.std()*10, o_0.std()))
def changeOxygenThreshold(O_threshold):
    global oxygen, oSize
    peakPointsO = (o_0>O_threshold).reshape(-1)
    if oxygen: tv.remove_representation(oxygen)
    oxygen = tv.add_representation('points', {'coordinates' : xyz[peakPointsO], 
                                   'sizes' : o_0.reshape(-1)[peakPointsO]*oSize, 
                                   'colors' : [0xAA0000]*np.sum(peakPointsO)})
    
@interact(H_threshold=(h_0.mean(), h_0.mean()+h_0.std()*10, h_0.std()))
def changeHydrogenThreshold(H_threshold):
    global hydrogen, hSize
    peakPointsH = (h_0>H_threshold).reshape(-1)
    if hydrogen: tv.remove_representation(hydrogen)
    hydrogen = tv.add_representation('points', {'coordinates' : xyz[peakPointsH], 
                                     'sizes' : h_0.reshape(-1)[peakPointsH]*hSize, 
                                     'colors' : [0xAAAAAA]*np.sum(peakPointsH)})

display.display(tv)

Let me add the electron density as well. But I will remove the explicit ions so that we can see the electron cloud.

In [13]:
oSize, hSize, nSize = 10, 10, 1
oxygen, hydrogen, n = None, None, None
tv = RepresentationViewer()
@interact(O_threshold=(o_0.mean(), o_0.mean()+o_0.std()*10, o_0.std()))
def changeOxygenThreshold(O_threshold=0.01251):
    global oxygen, oSize
    peakPointsO = (o_0>O_threshold).reshape(-1)
    if oxygen: tv.remove_representation(oxygen)
    oxygen = tv.add_representation('points', {'coordinates' : xyz[peakPointsO], 
                                   'sizes' : o_0.reshape(-1)[peakPointsO]*oSize, 
                                   'colors' : [0xAA0000]*np.sum(peakPointsO)})
    
@interact(H_threshold=(h_0.mean(), h_0.mean()+h_0.std()*10, h_0.std()))
def changeHydrogenThreshold(H_threshold=0.0194):
    global hydrogen, hSize
    peakPointsH = (h_0>H_threshold).reshape(-1)
    if hydrogen: tv.remove_representation(hydrogen)
    hydrogen = tv.add_representation('points', {'coordinates' : xyz[peakPointsH], 
                                     'sizes' : h_0.reshape(-1)[peakPointsH]*hSize, 
                                     'colors' : [0xAAAAAA]*np.sum(peakPointsH)})

def update_n(tv, n, size, lowerLim, upperLim):
    ind_n = np.logical_and(n_0>lowerLim, n_0<upperLim).reshape(-1)
    if not ind_n.any():
        return n
    if np.sum(ind_n)>10000:
        print("Too many n points:", np.sum(ind_n), '\nChange the limits!')
        return n
    else:
        print("Number of points for n:", np.sum(ind_n))
    if n: tv.remove_representation(n)
    return tv.add_representation('points', {'coordinates' : xyz[ind_n], 
                                            'sizes' : n_0.reshape(-1)[ind_n]*size, 
                                            'colors' : [0x00AA00]*len(xyz[ind_n]) })

@interact(nSize=(0.1,13,.1), 
          n_lowerLim=(0, 1, 0.001),
          n_upperLim=(0.001, 1, 0.001)
          )
def change_n_Size(nSize=1.4, n_lowerLim=0.042, n_upperLim=0.065):#, ln_oxy_size, ln_h_size, H_threshold):
    global n
    nSize = np.exp(nSize)
    n = update_n(tv, n, nSize, n_lowerLim, n_upperLim)

display.display(tv)

Number of points for n: 9333
